### A Learning Algorithm for Threshold Boolean Networks with Prescribed Fixed Points

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import time

# Binary sigmoid and derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(sigmoid_x):
    return sigmoid_x * (1 - sigmoid_x)

# Compute fixed points with hard-threshold updates
def compute_fixed_points(W, b):
    all_states = np.array(list(product([0, 1], repeat=13)), dtype=np.float32)
    preds_bin = ((all_states @ W + b) >= 0).astype(np.float32)
    fixed = [s for s, y in zip(all_states, preds_bin) if np.array_equal(s, y)]
    return np.array(fixed)

# Desired fixed points
desired_fp = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
], dtype=np.float32)

# Training parameters
epochs = 1000
lr = 0.1
lambda_bin = 0.1
lambda_l1 = 0.01
lambda_spurious = 5.0
eps = 1e-6
n = 13

# Result containers
results = []

# 30-run experiment
for run in range(30):
    print(f"Run {run+1}/30")
    W = np.random.randn(n, n) * 0.1
    b = np.zeros(n)

    start_time = time.time()

    for epoch in range(epochs):
        Z = desired_fp @ W + b
        Y_pred = sigmoid(Z)
        sig_der = sigmoid_derivative(Y_pred)

        err_pos = Y_pred - desired_fp
        grad_pos = (2 / len(desired_fp)) * desired_fp.T @ (err_pos * sig_der)
        db_pos = (2 / len(desired_fp)) * np.sum(err_pos * sig_der, axis=0)

        grad_bin = (lambda_bin / len(desired_fp)) * desired_fp.T @ ((1 - 2 * Y_pred) * sig_der)
        db_bin = (lambda_bin / len(desired_fp)) * np.sum((1 - 2 * Y_pred) * sig_der, axis=0)

        # Dynamically find spurious fixed points
        spurious_fp = compute_fixed_points(W, b)
        spurious_fp = [x for x in spurious_fp if not any(np.array_equal(x, f) for f in desired_fp)]

        grad_spurious = np.zeros_like(W)
        db_spurious = np.zeros_like(b)

        if spurious_fp:
            spurious_fp = np.array(spurious_fp)
            Z_spur = spurious_fp @ W + b
            Y_spur = sigmoid(Z_spur)
            sig_der_spur = sigmoid_derivative(Y_spur)
            err_spur = Y_spur - spurious_fp
            r = np.sum(err_spur**2, axis=1) + eps

            for i in range(len(spurious_fp)):
                outer = (err_spur[i] * sig_der_spur[i]) / (r[i]**2)
                grad_spurious += np.outer(spurious_fp[i], outer)
                db_spurious += outer
            grad_spurious *= -2 * lambda_spurious / len(spurious_fp)
            db_spurious *= -2 * lambda_spurious / len(spurious_fp)

        grad_l1 = lambda_l1 * np.sign(W)

        dW = grad_pos + grad_bin + grad_spurious + grad_l1
        db = db_pos + db_bin + db_spurious

        W -= lr * dW
        b -= lr * db

    # Final evaluation
    elapsed = time.time() - start_time
    final_fp = compute_fixed_points(W, b)
    matched = sum(any(np.array_equal(fp, x) for fp in desired_fp) for x in final_fp)
    spurious = len(final_fp) - matched

    results.append({
        "run": run + 1,
        "W": W,
        "b": b,
        "fixed_points": len(final_fp),
        "matched": matched,
        "spurious": spurious,
        "time_sec": elapsed
    })

# Summary table
summary = pd.DataFrame([{
    "run": r["run"],
    "fixed_points": r["fixed_points"],
    "matched": r["matched"],
    "spurious": r["spurious"],
    "time_sec": r["time_sec"]
} for r in results])

# Save results
summary.to_csv("tbn_experiment_summary.csv", index=False)
np.save("tbn_experiment_weights.npy", np.array([r["W"] for r in results]))
np.save("tbn_experiment_biases.npy", np.array([r["b"] for r in results]))

print("Experiment completed and saved.")

### Perceptron baseline

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import time
from sklearn.linear_model import Perceptron

# Desired fixed points
desired_fp = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
], dtype=np.float32)

# Compute fixed points using hard-thresholding
def compute_fixed_points(W, b):
    all_states = np.array(list(product([0, 1], repeat=13)), dtype=np.float32)
    preds_bin = ((all_states @ W.T + b) >= 0).astype(np.float32)
    fixed = [s for s, y in zip(all_states, preds_bin) if np.array_equal(s, y)]
    return np.array(fixed)

# Initialize parameters
n = 13
runs = 30
results = []

for run in range(runs):
    print(f"Run {run+1}/{runs}")
    start = time.time()

    W = np.zeros((n, n))
    b = np.zeros(n)

    # Train one perceptron per node
    for i in range(n):
        clf = Perceptron(max_iter=1000, tol=1e-4, random_state=None, fit_intercept=True)
        clf.fit(desired_fp, desired_fp[:, i])
        W[i, :] = clf.coef_
        b[i] = clf.intercept_

    elapsed = time.time() - start

    # Evaluate the resulting TBN
    final_fp = compute_fixed_points(W, b)
    matched = sum(any(np.array_equal(fp, x) for fp in desired_fp) for x in final_fp)
    spurious = len(final_fp) - matched

    results.append({
        "run": run + 1,
        "W": W.copy(),
        "b": b.copy(),
        "fixed_points": len(final_fp),
        "matched": matched,
        "spurious": spurious,
        "time_sec": elapsed
    })

# Summary DataFrame
summary = pd.DataFrame([{
    "run": r["run"],
    "fixed_points": r["fixed_points"],
    "matched": r["matched"],
    "spurious": r["spurious"],
    "time_sec": r["time_sec"]
} for r in results])

# Save outputs
summary.to_csv("tbn_baseline_perceptron_summary.csv", index=False)
np.save("tbn_baseline_W.npy", np.array([r["W"] for r in results]))
np.save("tbn_baseline_b.npy", np.array([r["b"] for r in results]))

print("Perceptron baseline experiment completed and saved.")


### Logistic Regression baseline

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import time
from sklearn.linear_model import LogisticRegression

# Desired fixed points
desired_fp = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
], dtype=np.float32)

# Compute fixed points using hard-thresholding
def compute_fixed_points(W, b):
    all_states = np.array(list(product([0, 1], repeat=13)), dtype=np.float32)
    logits = all_states @ W.T + b
    preds_bin = (logits >= 0).astype(np.float32)
    fixed = [s for s, y in zip(all_states, preds_bin) if np.array_equal(s, y)]
    return np.array(fixed)

# Parameters
n = 13
runs = 30
results = []

for run in range(runs):
    print(f"Logistic Regression Run {run+1}/{runs}")
    start = time.time()

    W = np.zeros((n, n))
    b = np.zeros(n)

    for i in range(n):
        clf = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
        clf.fit(desired_fp, desired_fp[:, i])
        W[i, :] = clf.coef_
        b[i] = clf.intercept_

    elapsed = time.time() - start

    final_fp = compute_fixed_points(W, b)
    matched = sum(any(np.array_equal(fp, x) for fp in desired_fp) for x in final_fp)
    spurious = len(final_fp) - matched

    results.append({
        "run": run + 1,
        "W": W.copy(),
        "b": b.copy(),
        "fixed_points": len(final_fp),
        "matched": matched,
        "spurious": spurious,
        "time_sec": elapsed
    })

# Save summary
summary = pd.DataFrame([{
    "run": r["run"],
    "fixed_points": r["fixed_points"],
    "matched": r["matched"],
    "spurious": r["spurious"],
    "time_sec": r["time_sec"]
} for r in results])

summary.to_csv("tbn_logreg_baseline_summary.csv", index=False)
np.save("tbn_logreg_W.npy", np.array([r["W"] for r in results]))
np.save("tbn_logreg_b.npy", np.array([r["b"] for r in results]))

print("Logistic Regression baseline completed and saved.")
